In [32]:
import pandas as pd
import duckdb as db
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from difflib import get_close_matches

plt.style.use('efrem.mplstyle')

df_sökande= pd.read_csv(
    "../data/2024_kurser/Behöriga_ansökare_per_region.csv",
    encoding="latin-1"
)
df = pd.read_excel("../data/2024_kurser/inkomna-ansokningar-2024-for-kurser.xlsx")

df_sökande.head()


,kön,utbildningsområde MYH,region (hemlän),Antal behöriga sökande 2020,Antal behöriga sökande 2021,Antal behöriga sökande 2022,Antal behöriga sökande 2023,Antal behöriga sökande 2024,Antal antagna vid start 2020,Antal antagna vid start 2021,Antal antagna vid start 2022,Antal antagna vid start 2023,Antal antagna vid start 2024
0,totalt,Data/It,Stockholms län,505,747,408,975,1523,291,390,303,756,1136
1,totalt,Data/It,Uppsala län,32,51,41,84,123,23,29,28,72,95
2,totalt,Data/It,Södermanlands län,15,28,19,34,65,9,17,13,25,52
3,totalt,Data/It,Östergötlands län,27,60,35,56,108,20,34,29,44,83
4,totalt,Data/It,Jönköpings län,12,28,15,38,69,9,20,13,27,56


In [33]:
# Ta bort kolumnen "kön"
df_transformed = df_sökande.drop(columns=["kön"])

df_transformed["utbildningsområde MYH"] = df_transformed["utbildningsområde MYH"].str.replace("MYH", "", regex=False).str.strip()

# Rensa alla kolumnnamn från överflödiga mellanslag i början/slutet
df_transformed.columns = df_transformed.columns.str.strip()


# Ta bort "MYH" från kolumnen "utbildningsområde MYH"
df_transformed.columns = [col.replace("MYH", "").strip() for col in df_transformed.columns]
df_transformed.columns = [col[:1].upper() + col[1:] for col in df_transformed.columns]


df_transformed.head()

,Utbildningsområde,Region (hemlän),Antal behöriga sökande 2020,Antal behöriga sökande 2021,Antal behöriga sökande 2022,Antal behöriga sökande 2023,Antal behöriga sökande 2024,Antal antagna vid start 2020,Antal antagna vid start 2021,Antal antagna vid start 2022,Antal antagna vid start 2023,Antal antagna vid start 2024
0,Data/It,Stockholms län,505,747,408,975,1523,291,390,303,756,1136
1,Data/It,Uppsala län,32,51,41,84,123,23,29,28,72,95
2,Data/It,Södermanlands län,15,28,19,34,65,9,17,13,25,52
3,Data/It,Östergötlands län,27,60,35,56,108,20,34,29,44,83
4,Data/It,Jönköpings län,12,28,15,38,69,9,20,13,27,56


In [34]:
df_top_omraden = db.query(
    """
    SELECT 
      "Sökt utbildningsområde" AS Utbildningsområde,
      SUM("Sökt antal platser 2024") AS År2024,
      SUM("Sökt antal platser 2025") AS År2025,
      SUM("Sökt antal platser 2025") - SUM("Sökt antal platser 2024") AS Tillväxt
    FROM df
    GROUP BY "Sökt utbildningsområde"
    ORDER BY Tillväxt DESC
    LIMIT 10
    """
).df().set_index("Utbildningsområde")

In [ ]:
from utils.constant import DARK_ORANGE, SADDLEBROWN, WHEAT_BROWN

ax = df_top_omraden[["År2024", "År2025"]].plot(
    kind="barh",
    figsize=(12, 6),
    color=[DARK_ORANGE, SADDLEBROWN],
)

ax.set_title("VÄXANDE UTBILDNINGSOMRÅDE I REGIONEN", loc="left", pad=20)
ax.set_xlabel("ANTAL SÖKANDE FÖR SKOLOR", loc="left")
ax.set_ylabel("UTBILDNINGSOMRÅDE", rotation= 0)
ax.yaxis.set_label_coords(-0.12, 1)

ax.patch.set_facecolor(WHEAT_BROWN)
ax.invert_yaxis()
ax.legend().remove()